In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
input_dir = '../input/m5-forecasting-accuracy/'
cal = pd.read_csv(f'{input_dir}calendar.csv')
sales_tv = pd.read_csv(f'{input_dir}sales_train_validation.csv')
sell_p = pd.read_csv(f'{input_dir}sell_prices.csv')


In [ ]:
cal.head()

In [ ]:
sales_tv.head()


In [ ]:
sell_p.head()


In [ ]:
sns.heatmap(cal.isnull(),xticklabels= False,yticklabels = False,cmap ='coolwarm')

In [ ]:
len(sales_tv['item_id'].unique())

In [ ]:
d_cols = [c for c in sales_tv.columns if 'd_' in c]
sales_tv.loc[sales_tv['id']=='HOBBIES_1_001_CA_1_validation']\
.set_index('id')[d_cols]\
.T\
.plot(figsize=(15,4)
     ,title = 'HOBBIES_1_001_CA_1_validation')
plt.legend('')


    



X axis labels are not easy for the naive users to understand so we merge the x axis labels with the real dates so its easy to understand and we can find weekly and annual trends.

In [ ]:
cal[['d','date','event_name_1','event_name_2',
     'event_type_1','event_type_2', 'snap_CA']].head()

In [ ]:
example = sales_tv.loc[sales_tv['id'] == 'FOODS_3_090_CA_3_validation'][d_cols].T
example = example.rename(columns={8412:'FOODS_3_090_CA_3'})
example = example.reset_index().rename(columns={'index': 'd'}) 
example = example.merge(cal, how='left', validate='1:1')
example.set_index('date')['FOODS_3_090_CA_3'] \
    .plot(figsize=(15, 5),
          title='FOODS_3_090_CA_3 sales by actual sale dates')
plt.show()


example2 = sales_tv.loc[sales_tv['id'] == 'HOBBIES_1_234_CA_3_validation'][d_cols].T
example2 = example2.rename(columns={6324:'HOBBIES_1_234_CA_3'}) 
example2 = example2.reset_index().rename(columns={'index': 'd'}) 
example2 = example2.merge(cal, how='left', validate='1:1')

example3 = sales_tv.loc[sales_tv['id'] == 'HOUSEHOLD_1_118_CA_3_validation'][d_cols].T
example3 = example3.rename(columns={6776:'HOUSEHOLD_1_118_CA_3'}) 
example3 = example3.reset_index().rename(columns={'index': 'd'}) 
example3 = example3.merge(cal, how='left', validate='1:1')

In [ ]:
examples = ['FOODS_3_090_CA_3','HOBBIES_1_234_CA_3','HOUSEHOLD_1_118_CA_3']
example_df = [example, example2, example3]
for i in [0,1,2]:
    fig , (ax1, ax2,ax3) = plt.subplots(1,3,figsize=(15,4))
    example_df[i].groupby('wday').mean()[examples[i]]\
    .plot(kind = 'line',
         title = 'average sales : day of week',
         linewidth=4,
         color = 'brown',
         ax = ax1)
    example_df[i].groupby('month').mean()[examples[i]]\
    .plot(kind = 'line',
         title = 'average sales : month',
         linewidth=5,
         color = 'blue',
         ax = ax2)
    example_df[i].groupby('wday').mean()[examples[i]]\
    .plot(kind = 'line',
         title = 'average sales :year',
         linewidth=5,
         color = 'red',
         ax = ax3)
    fig.suptitle(f'trends of :{examples[i]}',
                size = 20,
                 y= 1.1
                 
                )
    plt.tight_layout()
    plt.show()

Observing the sales of every items .

In [ ]:
twenty_examples = sales_tv.sample(20, random_state=529) \
        .set_index('id')[d_cols] \
    .T \
    .merge(cal.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date')

In [ ]:
fig, axs = plt.subplots(10, 2, figsize=(15, 20))
axs = axs.flatten()
ax_idx = 0
for item in twenty_examples.columns:
    twenty_examples[item].plot(title=item,

                              ax=axs[ax_idx])
    ax_idx += 1
plt.tight_layout()
plt.show()

In [ ]:
sales_tv['cat_id'].unique()

In [ ]:
sales_tv.groupby('cat_id').count()['id']\
   .sort_values()\
   .plot(kind = 'barh',figsize=(15,6))\
   


In [ ]:
past_sales = sales_tv.set_index('id')[d_cols] \
    .T \
    .merge(cal.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date')


for i in sales_tv['cat_id'].unique():
    items_col = [c for c in past_sales.columns if i in c]
    past_sales[items_col] \
        .sum(axis=1) \
        .plot(figsize=(15, 5),
              alpha=0.8,
              title='Total Sales by Item Type')
plt.legend(sales_tv['cat_id'].unique())
plt.show()

In [ ]:
past_sales_clipped = past_sales.clip(0,1)
for i in sales_tv['cat_id'].unique():
    items_col = [c for c in past_sales.columns if i in c]
    (past_sales_clipped[items_col] \
        .mean(axis=1) * 100) \
        .plot(figsize=(15, 5),
              alpha=0.8,
              title='Inventory Sale Percentage by Date',
              style='.')
    plt.ylabel('% of inventory with atleast 1 sale')
    plt.legend(sales_tv['cat_id'].unique())
    

In [ ]:
store_list = sell_p['store_id'].unique()
for s in store_list:
    store_items = [c for c in past_sales.columns if s in c]
    past_sales[store_items] \
        .sum(axis=1) \
        .rolling(90).mean() \
        .plot(figsize=(15, 5),
              alpha=0.8,
              title='Rolling 90 Day Average Total Sales (10 stores)')
plt.legend(store_list)

In [ ]:
sell_p['Category'] = sell_p['item_id'].str.split('_', expand=True)[0]
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
i = 0
for cat, d in sell_p.groupby('Category'):
    ax = d['sell_price'].apply(np.log1p) \
        .plot(kind='hist',
                         bins=20,
                         title=f'Distribution of {cat} prices',
                         ax=axs[i]
                                         )
    ax.set_xlabel('Log(price)')
    i += 1
plt.tight_layout()

**References : https://www.kaggle.com/robikscube/m5-forecasting-starter-data-exploration**